In [1]:
# %%
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='4'

import torch
import torchaudio
import scipy.io.wavfile as wf
import numpy as np
import base64
import hashlib
import json 
import time
import uuid
import requests
import scipy
from speechbrain.pretrained import EncoderClassifier
import random
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")
# def dvector_extraction(wav_tensor1):
#     transform = torchaudio.transforms.MFCC(sample_rate = sample_rate,n_mfcc = 40, melkwargs={"n_fft": 1024, "hop_length": 128, "n_mels": 64, "center": False},)
#     mel_tensor1 = transform(wav_tensor1)[0].T
#     emb_tensor1 = dvector.embed_utterance(mel_tensor1)
#     return emb_tensor1
def xvector_extraction(wav_ts):
    return classifier.encode_batch(wavs=wav_ts)
def SNR(wav_tensor1, noise_tensor):
    temp1 = wav_tensor1
    temp2 = noise_tensor
    temp2 = temp2.detach().numpy()
    temp1 = temp1.numpy()
    ans = 20*np.log10(np.linalg.norm(temp1, ord=2)/np.linalg.norm(temp2, ord=2))
    return ans
# def getSign(timestamp, nonce):
#     hs = hashlib.sha256()
#     appkey = "zvpcvm5hxib3jz3vt2jshqxjndcywwo2qnx7s6iy"
#     secret = '3c8a61e2bdb0ee2d0af74814142ba2ee'
#     hs.update((appkey + timestamp + secret + nonce).encode('utf-8'))
#     signature = hs.hexdigest().upper()
#     return signature
# def identifyFeatureByGroupId(confirmFeatureFileName):
#     identify_feature = open(confirmFeatureFileName, 'rb').read()
#     # 声纹base64字符串
#     audio_data = base64.b64encode(identify_feature)
#     timestamp = str(int(time.time() * 1000))
#     nonce = str(uuid.uuid1()).replace('-', '')
#     sign = getSign(timestamp, nonce)
#     headers = {"Content-Type": "application/json"}
#     appkey = "zvpcvm5hxib3jz3vt2jshqxjndcywwo2qnx7s6iy"
#     groupId = '113'
#     host = 'https://ai-vpr.hivoice.cn'
#     identifyFeatureByGroupIdEndPoint = '/vpr/v1/identifyFeatureByGroupId'
#     identify_feature_param = {
#         "appkey": appkey,
#         "timestamp": timestamp,
#         "nonce": nonce,
#         "sign": sign,
#         "groupId": groupId,
#         "topN": 10,
#         "audioData": audio_data.decode(),
#         "audioSampleRate": 16000,
#         "audioFormat": "wav"
#     }
#     #print('identify_feature_param', identify_feature_param)
#     identify_feature_resp = requests.post(url=host + identifyFeatureByGroupIdEndPoint,
#                                           data=json.dumps(identify_feature_param),
#                                           headers=headers)
#     identify_feature_result = json.loads(identify_feature_resp.content)
#     return identify_feature_result['data'][0]['featureInfo'], identify_feature_result['data'][0]['score'],identify_feature_result['data'][1]['featureInfo'], identify_feature_result['data'][1]['score']
def loss_function(AdvTensor, TarTensor,noise_tensor):
    return torch.norm(AdvTensor - TarTensor) +  0.5 * torch.norm(noise_tensor, float('inf'))

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
path1 = "/mnt/data/Chenpinji/attck_xiaomi/origin/"
path2 = "/mnt/data/Chenpinji/attck_xiaomi/target/"
outfilepath = '/mnt/data/Chenpinji/attck_xiaomi/result/'
for j in range(len(os.listdir(path2))):
    filename2 = path2 + sorted(os.listdir(path2), key = lambda fn: list(map(ord, fn)))[j]
    print(filename2)#target
    for i in range(len(os.listdir(path1))):
        file = os.listdir(path1)
        filepath1 = path1 + sorted(file,key = lambda fn: list(map(ord, fn)))[i]
        for i1 in range(len(os.listdir(filepath1))):
            audiofile = os.listdir(filepath1)
            filename1 = filepath1 + '/'+ sorted(audiofile, key = lambda fn: list(map(ord, fn)))[i1]
            print(filename1)#origin
            outpath = outfilepath + sorted(os.listdir(outfilepath), key = lambda fn: list(map(ord, fn)))[j]
            outfile = outpath + '/' + sorted(os.listdir(outfilepath), key = lambda fn: list(map(ord, fn)))[j][0:3]  +'_'+'SNR'+'_'+sorted(audiofile, key = lambda fn: list(map(ord, fn)))[i1]
            print(outfile)
            print()


In [5]:
path1 = "/mnt/data/Chenpinji/attck_xiaomi/origin/"
filename2 = "/mnt/data/Chenpinji/attck_xiaomi/target/zyp_2.wav"
outfilepath = '/mnt/data/Chenpinji/attck_xiaomi/result/zyp/'
for i in range(0, len(os.listdir(path1))):
    file = os.listdir(path1)
    filename1 = path1 + sorted(file,key = lambda fn: list(map(ord, fn)))[i]
    if sorted(file,key = lambda fn: list(map(ord, fn)))[i] == "zyp_2.wav":
        continue
    wav_tensor1, sample_rate = torchaudio.load(filename1,normalize = False)
    wav_tensor2, sample_rate = torchaudio.load(filename2,normalize = False)
    noise_tensor = torch.normal(mean=200.,std=50.,size=(1,len(wav_tensor1[0])))
    wav_tensor1 = wav_tensor1.to(torch.float)
    wav_tensor2 = wav_tensor2.to(torch.float)
    noise_tensor.to(device=device)
    noise_tensor.requires_grad = True
    optimizer = torch.optim.Adam([noise_tensor],lr = 3)
    # epochs = random.randint(500,1000)
    epochs = 1100
    for epoch in range(epochs):
        AdvTensor = xvector_extraction(wav_tensor1+noise_tensor)
        TarTensor = xvector_extraction(wav_tensor2)
        loss = loss_function(AdvTensor, TarTensor, noise_tensor)
    #     print("epoch={} loss={}".format(epoch, loss))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0 or epoch == epochs - 1:
            print("epoch={} loss={}".format(epoch, loss))
            snr = SNR(wav_tensor1, noise_tensor)
            print("SNR:",snr)
            audio_name = outfilepath  + 'zy['  +'_'+str(snr)+'_'+ sorted( file , key = lambda fn: list(map(ord, fn)))[i]
            temp = wav_tensor1 + noise_tensor 
            temp = temp.detach().numpy()
            scaled = np.array(np.clip(np.round(temp[0]),-2 ** 15, 2 ** 15 - 1), dtype = np.int16)
            #print("epoch={} loss={}".format(epoch, loss))
            print(audio_name)
            wf.write(audio_name, 44100 ,scaled)
        #speaker1,score1,speaker2,score2 = identifyFeatureByGroupId(audio_name)
        # print('classified as:',speaker1,'Score is:',score1)
        # print('second place is:',speaker2, 'Score is:',score2)
        # print()

epoch=0 loss=316.463134765625
SNR: 30.257291793823242
/mnt/data/Chenpinji/attck_xiaomi/result/zyp/zy[_30.257291793823242_cxz_2.wav
epoch=50 loss=229.85794067382812
SNR: 30.060677528381348
/mnt/data/Chenpinji/attck_xiaomi/result/zyp/zy[_30.060677528381348_cxz_2.wav
epoch=100 loss=245.79080200195312
SNR: 29.949519634246826
/mnt/data/Chenpinji/attck_xiaomi/result/zyp/zy[_29.949519634246826_cxz_2.wav
epoch=150 loss=253.18325805664062
SNR: 29.88954782485962
/mnt/data/Chenpinji/attck_xiaomi/result/zyp/zy[_29.88954782485962_cxz_2.wav
epoch=200 loss=258.246337890625
SNR: 29.849493503570557
/mnt/data/Chenpinji/attck_xiaomi/result/zyp/zy[_29.849493503570557_cxz_2.wav
epoch=250 loss=257.9100341796875
SNR: 29.818270206451416
/mnt/data/Chenpinji/attck_xiaomi/result/zyp/zy[_29.818270206451416_cxz_2.wav
epoch=300 loss=259.08721923828125
SNR: 29.792308807373047
/mnt/data/Chenpinji/attck_xiaomi/result/zyp/zy[_29.792308807373047_cxz_2.wav
epoch=350 loss=257.9142150878906
SNR: 29.770960807800293
/mnt/dat